In [1]:
import pyspark
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import StructField, StructType, IntegerType, DateType, StringType, TimestampType, DoubleType, LongType
import pyspark.sql.functions as F
warnings.filterwarnings("ignore")

In [2]:
spark = SparkSession.builder.master("spark://zoomcamp.europe-west6-a.c.nyc-taxi-pipeline.internal:7077")\
                    .appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 16:30:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
df = spark.read.option("header", "true").parquet("./data/stage/green_tripdata_2020-01_partitioned")

In [13]:
df = df.withColumn("lpep_pickup_date", F.to_date(df["lpep_pickup_datetime"]))\
       .withColumn("lpep_dropoff_date", F.to_date(df["lpep_dropoff_datetime"]))

In [14]:
location_lookup = spark.read.option("header", "true").csv("./data/taxi+_zone_lookup.csv")

In [15]:
merged_df = df.alias("base").join(location_lookup.alias("a"),
                                  F.col("base.PULocationID") == F.col("a.LocationID"),
                                  how = "left")\
                            .join(location_lookup.alias("b"),
                                  F.col("base.DOLocationID") == F.col("b.LocationID"),
                                  how = "left")\
                            .select(F.col("a.Zone").alias("PUZone"), F.col("b.Zone").alias("DOZone"), "trip_distance")

In [16]:
merged_df.repartition(4).write.parquet("./data/trip_loc_dist/green_202001")

23/03/04 16:49:15 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
23/03/04 16:49:15 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:218)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:923)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce